In [1]:
#mengambil konten dari Wikipedia dalam format yang rumit dan mengonversinya menjadi format teks yang lebih mudah untuk analisis lebih lanjut.
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2024-11-01 14:56:58,984: INFO: running C:\Users\annis\anaconda3\Lib\site-packages\ipykernel_launcher.py -f C:\Users\annis\AppData\Roaming\jupyter\runtime\kernel-8ac7bac8-e2f0-4c73-8076-e04ff72b4aec.json
2024-11-01 14:58:04,132: INFO: Saved 10000 articles
2024-11-01 14:58:42,465: INFO: Saved 20000 articles
2024-11-01 14:59:08,934: INFO: Saved 30000 articles
2024-11-01 14:59:32,775: INFO: Saved 40000 articles
2024-11-01 14:59:57,623: INFO: Saved 50000 articles
2024-11-01 15:00:34,020: INFO: Saved 60000 articles
2024-11-01 15:01:04,033: INFO: Saved 70000 articles
2024-11-01 15:01:30,153: INFO: Saved 80000 articles
2024-11-01 15:02:11,815: INFO: Saved 90000 articles
2024-11-01 15:02:39,392: INFO: Saved 100000 articles
2024-11-01 15:03:04,248: INFO: Saved 110000 articles
2024-11-01 15:03:26,936: INFO: Saved 120000 articles
2024-11-01 15:04:05,673: INFO: Saved 130000 articles
2024-11-01 15:04:31,699: INFO: Saved 140000 articles
2024-11-01 15:04:50,698: INFO: Saved 150000 articles
2024-11-01 

In [2]:
#elatih model Word2Vec menggunakan teks yang diambil dari Wikipedia dalam bahasa Indonesia, kemudian menyimpan model yang terlatih ke dalam format file teks.
#Model Word2Vec digunakan untuk menghasilkan representasi vektor dari kata-kata, yang dapat digunakan dalam berbagai aplikasi pemrosesan bahasa alami (NLP)
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2024-11-01 15:23:58,763: INFO: running C:\Users\annis\anaconda3\Lib\site-packages\ipykernel_launcher.py -f C:\Users\annis\AppData\Roaming\jupyter\runtime\kernel-8ac7bac8-e2f0-4c73-8076-e04ff72b4aec.json
2024-11-01 15:23:58,795: INFO: collecting all words and their counts
2024-11-01 15:23:58,807: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-11-01 15:24:04,027: INFO: PROGRESS: at sentence #10000, processed 11193422 words, keeping 360573 word types
2024-11-01 15:24:08,072: INFO: PROGRESS: at sentence #20000, processed 18100685 words, keeping 499518 word types
2024-11-01 15:24:10,870: INFO: PROGRESS: at sentence #30000, processed 22961169 words, keeping 599591 word types
2024-11-01 15:24:14,040: INFO: PROGRESS: at sentence #40000, processed 27547646 words, keeping 686069 word types
2024-11-01 15:24:17,975: INFO: PROGRESS: at sentence #50000, processed 32493667 words, keeping 785060 word types
2024-11-01 15:24:20,148: INFO: PROGRESS: at sentence #60000, proce

In [7]:
#menggunakan model Word2Vec untuk menemukan kesamaan kata, menghitung kemiripan antar kata, serta melakukan analisa analogi berbasis vektor. 
import gensim
# import gensim.models.keyedvectors as word2vec
# model = gensim.models.Word2Vec.load(namaFileModel)
# model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

namaFileModel = "w2vec_wiki_id300_0.txt"

sim = model.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta:{}".format(sim))
sim = model.wv.most_similar("Bandung")
print("10 kata terdekat dari Bandung:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Jakarta:[('Ragunan', 0.6153457164764404), ('Kemayoran', 0.6127204895019531), ('Warteg', 0.6067242622375488), ('Surabaya', 0.6009742617607117), ('Kemang', 0.5919972658157349), ('Jagakarsa', 0.5910428166389465), ('Senayan', 0.5861063003540039), ('Ciputat', 0.5783038139343262), ('Cilandak', 0.5774096846580505), ('Palmerah', 0.5746968388557434)]
10 kata terdekat dari Bandung:[('Cimahi', 0.7192736864089966), ('Bogor', 0.7001960873603821), ('Tasikmalaya', 0.6768466234207153), ('Garut', 0.6581134796142578), ('Sukabumi', 0.642021656036377), ('Baleendah', 0.6318262815475464), ('Jatinangor', 0.6172360181808472), ('Purwakarta', 0.6154208779335022), ('Cianjur', 0.6016227006912231), ('Sumedang', 0.5990661382675171)]
Kedekatan Yogyakarta-Surakarta: 0.7897931933403015
Kedekatan Yogyakarta-Semarang: 0.646669864654541
makanan-rendang, minuman-?: [('wedang', 0.8733793169947867), ('Minuman', 0.8697142126003948), ('racikan', 0.8573810726746108), ('berkuah', 0.8447060618038803), ('kal